In [20]:
# code setup
@show versioninfo()
@show pwd()
import Pkg; Pkg.activate("..")
using Symbolics
using Symbolics: jacobian, derivative
using BenchmarkTools

Julia Version 1.7.2
Commit bf53498635 (2022-02-06 15:21 UTC)
Platform Info:
  OS: Linux (x86_64-pc-linux-gnu)
  CPU: AMD Ryzen 9 3950X 16-Core Processor
  WORD_SIZE: 64
  LIBM: libopenlibm
  LLVM: libLLVM-12.0.1 (ORCJIT, znver2)
versioninfo() = nothing
pwd() = "/home/mantas/.julia/dev/PRONTO.jl/scripts"


  Activating project at `~/.julia/dev/PRONTO.jl`


In [29]:
# to compare against PRONTO for Dummies

# dynamics
g = 9.81;
L = 2;

# f(x,u,t) = f(x(t), u(t))
f(x,u) = [x[2];   g/L*sin(x[1])-u*cos(x[1])/L]
# f(x,u) = [x[1]; x[2]]
@variables x[1:2] u
f(x,u)

2-element Vector{Num}:
 x[2]
    4.905sin(x[1]) - (1//2)*u*cos(x[1])

In [30]:
for g in f(x,u)
    for x in x
        println("d$g/d$x")
    end
    # println(g)
end



dx[2]/dx[1]
dx[2]/dx[2]
d4.905sin(x[1]) - (1//2)*u*cos(x[1])/dx[1]
d4.905sin(x[1]) - (1//2)*u*cos(x[1])/dx[2]


In [47]:
fx_sym = hcat(map(1:2) do i
    map(1:2) do j
        derivative(f(x,u)[j],x[i])
    end
end...)

2×2 Matrix{Num}:
 0                                    1
 4.905cos(x[1]) + (1//2)*u*sin(x[1])  0

In [50]:
fx = eval(build_function(fx_sym, x, u)[1])

#48 (generic function with 1 method)

In [51]:
fx(x,u)

2×2 Matrix{Num}:
 0                                    1
 4.905cos(x[1]) + (1//2)*u*sin(x[1])  0

In [ ]:

# incremental cost
Q = eye(2);
R = 1;

l(x,u,t) = l(x(t), u(t))
l(x,u) = 1/2*x'*Q*x + 1/2*R*u^2


# states/allocations
# x: [2,1]xT
# u: 1xT